<a href="https://colab.research.google.com/github/tathycosta/DDL_Clinica_veterinaria/blob/main/Exercicios_DDL_clinica_veterinaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mysql-connector-python

In [ ]:
import mysql.connector
import pandas as pd

In [ ]:
host='34.173.32.71'
user='root'
password=''
database='clinica_veterinaria'

In [ ]:
!curl ipecho.net/plain

34.106.204.130

In [ ]:
def consulta(query, tabela):
  connection = mysql.connector.connect(
      host = host,
      user = user,
      password = password,
      database = database
  )
  cursor = connection.cursor()

  try:
    cursor.execute(query)
    result = cursor.fetchall()
    globals()[tabela] = pd.DataFrame(result, columns=cursor.column_names)
    display(globals()[tabela])
  finally:
    cursor.close()
    connection.close()

In [ ]:
def ddl(comando):
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    cursor = connection.cursor()

    try:
        cursor.execute(comando)
        connection.commit()  # Confirma as mudanças no banco
        print(f"Comando DDL executado com sucesso:\n{comando}")
    except mysql.connector.Error as err:
        print(f"Erro ao executar o comando DDL: {err}")
    finally:
        cursor.close()
        connection.close()

In [ ]:
consulta('''
SHOW TABLES;
''','tabelas')

,Tables_in_clinica_veterinaria
0,Animais
1,Consultas
2,Funcionarios
3,Pagamentos
4,Servicos
5,Tratamentos
6,Tutores


In [ ]:
consulta('''
DESC Animais;
''','')

,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,
1,nome,varchar(100),NO,,None,
2,especie,varchar(50),NO,,None,
3,raca,varchar(50),YES,,None,
4,data_nascimento,date,YES,,None,
5,peso,"decimal(5,2)",YES,,None,
6,id_tutor,int,YES,MUL,None,


Criando as tabelas

In [ ]:
ddl('''
CREATE TABLE Tutores (
    id INT PRIMARY KEY,
    nome VARCHAR(100) NOT NULL,
    telefone VARCHAR(15),
    email VARCHAR(100) UNIQUE,
    endereco VARCHAR(200),
    data_cadastro DATE
);
''')

ddl('''
CREATE TABLE Animais (
    id INT PRIMARY KEY,
    nome VARCHAR(100) NOT NULL,
    especie VARCHAR(50) NOT NULL,
    raca VARCHAR(50),
    data_nascimento DATE,
    peso DECIMAL(5, 2),
    id_tutor INT,
    FOREIGN KEY (id_tutor) REFERENCES Tutores(id)
);
''')

ddl('''
CREATE TABLE Funcionarios (
    id INT PRIMARY KEY,
    nome VARCHAR(100) NOT NULL,
    cargo VARCHAR(50) NOT NULL,
    email VARCHAR(100) UNIQUE,
    data_admissao DATE
);
''')

ddl('''
CREATE TABLE Servicos (
    id INT PRIMARY KEY,
    nome VARCHAR(100) NOT NULL,
    descricao TEXT,
    preco DECIMAL(10, 2) NOT NULL
);
''')

ddl('''
CREATE TABLE Consultas (
    id INT PRIMARY KEY,
    data_consulta DATETIME NOT NULL,
    id_animal INT,
    id_funcionario INT,
    id_servico INT,
    observacoes VARCHAR(500),
    id_veterinario INT,
    FOREIGN KEY (id_animal) REFERENCES Animais(id),
    FOREIGN KEY (id_funcionario) REFERENCES Funcionarios(id),
    FOREIGN KEY (id_servico) REFERENCES Servicos(id),
    FOREIGN KEY (id_veterinario) REFERENCES Funcionarios(id)
);
''')

ddl('''
CREATE TABLE Tratamentos (
    id INT PRIMARY KEY,
    id_consulta INT,
    medicamento VARCHAR(100) NOT NULL,
    dosagem VARCHAR(50),
    frequencia VARCHAR(50),
    duracao VARCHAR(50),
    FOREIGN KEY (id_consulta) REFERENCES Consultas(id)
);
''')

ddl('''
CREATE TABLE Pagamentos (
    id INT PRIMARY KEY,
    id_consulta INT,
    valor DECIMAL(10, 2) NOT NULL,
    data_pagamento DATE,
    metodo_pagamento VARCHAR(50),
    FOREIGN KEY (id_consulta) REFERENCES Consultas(id)
);
''')

Comando DDL executado com sucesso:

CREATE TABLE Tutores (
    id INT PRIMARY KEY,  
    nome VARCHAR(100) NOT NULL,
    telefone VARCHAR(15),
    email VARCHAR(100) UNIQUE,
    endereco VARCHAR(200),
    data_cadastro DATE
);

Comando DDL executado com sucesso:

CREATE TABLE Animais (
    id INT PRIMARY KEY,  
    nome VARCHAR(100) NOT NULL,
    especie VARCHAR(50) NOT NULL,
    raca VARCHAR(50),
    data_nascimento DATE,
    peso DECIMAL(5, 2),
    id_tutor INT,
    FOREIGN KEY (id_tutor) REFERENCES Tutores(id)
);

Comando DDL executado com sucesso:

CREATE TABLE Funcionarios (
    id INT PRIMARY KEY,  
    nome VARCHAR(100) NOT NULL,
    cargo VARCHAR(50) NOT NULL,
    email VARCHAR(100) UNIQUE,
    data_admissao DATE
);

Comando DDL executado com sucesso:

CREATE TABLE Servicos (
    id INT PRIMARY KEY,  
    nome VARCHAR(100) NOT NULL,
    descricao TEXT,
    preco DECIMAL(10, 2) NOT NULL
);

Comando DDL executado com sucesso:

CREATE TABLE Consultas (
    id INT PRIMARY KEY,  
  

Inserção de dados (INSERT)

In [ ]:
# Inserir Tutores
ddl('''
INSERT INTO Tutores (id, nome, telefone, email, endereco, data_cadastro) VALUES
(1, 'Ana Clara', '(32) 9999-8888', 'ana.clara@email.com', 'Rua das Flores, 123', '2023-01-10'),
(2, 'Carlos Eduardo', '(32) 7777-6666', 'carlos.eduardo@email.com', 'Avenida Paulista, 456', '2023-02-15'),
(3, 'Mariana Silva', '(32) 5555-4444', 'mariana.silva@email.com', 'Rua dos Pinheiros, 789', '2023-03-20'),
(4, 'Pedro Henrique', '(32) 3333-2222', 'pedro.henrique@email.com', 'Alameda Santos, 101', '2023-04-25'),
(5, 'Juliana Costa', '(32) 1111-0000', 'juliana.costa@email.com', 'Rua Augusta, 202', '2023-05-30');
''')

# Inserir Animais
ddl('''
INSERT INTO Animais (id, nome, especie, raca, data_nascimento, peso, id_tutor) VALUES
(1, 'Rex', 'Cachorro', 'Labrador', '2018-05-10', 30.5, 1),
(2, 'Mimi', 'Gato', 'Siamês', '2019-03-15', 4.2, 2),
(3, 'Thor', 'Cachorro', 'Bulldog', '2020-07-22', 25.0, 3),
(4, 'Luna', 'Gato', 'Persa', '2017-11-30', 5.0, 4),
(5, 'Bella', 'Cachorro', 'Poodle', '2021-01-05', 8.7, 5);
''')

# Inserir Funcionários
ddl('''
INSERT INTO Funcionarios (id, nome, cargo, email, data_admissao) VALUES
(1, 'Dr. João Silva', 'Veterinário', 'joao.silva@clinica.com', '2020-01-15'),
(2, 'Dra. Maria Souza', 'Veterinária', 'maria.souza@clinica.com', '2019-05-20'),
(3, 'Carlos Oliveira', 'Atendente', 'carlos.oliveira@clinica.com', '2021-03-10'),
(4, 'Fernanda Lima', 'Recepcionista', 'fernanda.lima@clinica.com', '2022-07-01'),
(5, 'Ricardo Santos', 'Auxiliar Veterinário', 'ricardo.santos@clinica.com', '2023-02-12');
''')

# Inserir Serviços
ddl('''
INSERT INTO Servicos (id, nome, descricao, preco) VALUES
(1, 'Consulta Geral', 'Consulta de rotina para avaliação de saúde', 150.00),
(2, 'Vacinação', 'Aplicação de vacinas', 80.00),
(3, 'Cirurgia', 'Procedimento cirúrgico', 500.00),
(4, 'Exame de Sangue', 'Coleta e análise de sangue', 120.00),
(5, 'Banho e Tosa', 'Serviço de higiene e estética', 60.00);
''')

# Inserir Consultas
ddl('''
INSERT INTO Consultas (id, data_consulta, id_animal, id_funcionario, id_servico, observacoes, id_veterinario) VALUES
(1, '2023-10-25 10:00:00', 1, 1, 1, 'Animal apresentou febre e falta de apetite.', 1),
(2, '2023-10-26 14:30:00', 2, 2, 2, 'Aplicação da vacina V8.', 2),
(3, '2023-10-27 09:15:00', 3, 1, 3, 'Cirurgia de castração realizada com sucesso.', 1),
(4, '2023-10-28 11:00:00', 4, 2, 4, 'Exame de sangue para check-up anual.', 2),
(5, '2023-10-29 16:45:00', 5, 3, 5, 'Banho e tosa completo.', 3);
''')

# Inserir Tratamentos
ddl('''
INSERT INTO Tratamentos (id, id_consulta, medicamento, dosagem, frequencia, duracao) VALUES
(1, 1, 'Doxiciclina', '1 comprimido', 'A cada 12 horas', '7 dias'),
(2, 2, 'Vacina V8', '1 dose', 'Única', '1 dia'),
(3, 3, 'Metadona', '1 comprimido', 'A cada 8 horas', '3 dias'),
(4, 4, 'Promun dog tabs', '1 comprimido', 'Diário', '30 dias'),
(5, 5, 'Shampoo de tratamento', 'Uso tópico', 'Durante o banho', '30 dias');
''')

# Inserir Pagamentos
ddl('''
INSERT INTO Pagamentos (id, id_consulta, valor, data_pagamento, metodo_pagamento) VALUES
(1, 1, 150.00, '2023-10-25', 'Cartão de Crédito'),
(2, 2, 80.00, '2023-10-26', 'Dinheiro'),
(3, 3, 500.00, '2023-10-27', 'PIX'),
(4, 4, 120.00, '2023-10-28', 'Cartão de Débito'),
(5, 5, 60.00, '2023-10-29', 'Dinheiro');
''')


Comando DDL executado com sucesso:

INSERT INTO Tutores (id, nome, telefone, email, endereco, data_cadastro) VALUES
(1, 'Ana Clara', '(32) 9999-8888', 'ana.clara@email.com', 'Rua das Flores, 123', '2023-01-10'),
(2, 'Carlos Eduardo', '(32) 7777-6666', 'carlos.eduardo@email.com', 'Avenida Paulista, 456', '2023-02-15'),
(3, 'Mariana Silva', '(32) 5555-4444', 'mariana.silva@email.com', 'Rua dos Pinheiros, 789', '2023-03-20'),
(4, 'Pedro Henrique', '(32) 3333-2222', 'pedro.henrique@email.com', 'Alameda Santos, 101', '2023-04-25'),
(5, 'Juliana Costa', '(32) 1111-0000', 'juliana.costa@email.com', 'Rua Augusta, 202', '2023-05-30');

Comando DDL executado com sucesso:

INSERT INTO Animais (id, nome, especie, raca, data_nascimento, peso, id_tutor) VALUES
(1, 'Rex', 'Cachorro', 'Labrador', '2018-05-10', 30.5, 1),
(2, 'Mimi', 'Gato', 'Siamês', '2019-03-15', 4.2, 2),
(3, 'Thor', 'Cachorro', 'Bulldog', '2020-07-22', 25.0, 3),
(4, 'Luna', 'Gato', 'Persa', '2017-11-30', 5.0, 4),
(5, 'Bella', 'Cachor

In [ ]:
ddl('''
UPDATE Animais SET id_tutor = NULL WHERE id_tutor = 2 AND id_tutor=4;
''')

Comando DDL executado com sucesso:

UPDATE Animais SET id_tutor = NULL WHERE id_tutor = 2 AND id_tutor=4;



In [ ]:
consulta('''
SHOW TABLES;
''','')

,Tables_in_clinica_veterinaria
0,Animais
1,Consultas
2,Funcionarios
3,Servicos
4,Tutores


In [ ]:
consulta('''
SELECT * FROM Animais;
''','')

,id,nome,especie,raca,data_nascimento,peso,id_tutor
0,1,Rex,Cachorro,Labrador,2018-05-10,30.50,1
1,2,Mimi,Gato,Siamês,2019-03-15,4.20,2
2,3,Thor,Cachorro,Bulldog,2020-07-22,25.00,3
3,4,Luna,Gato,Persa,2017-11-30,5.00,4
4,5,Bella,Cachorro,Poodle,2021-01-05,8.70,5


In [ ]:
consulta('''
SELECT * FROM Funcionarios;
''','')

,id,nome,cargo,email,data_admissao
0,1,Dr. João Silva,Veterinário,joao.silva@clinica.com,2020-01-15
1,2,Dra. Maria Souza,Veterinária,maria.souza@clinica.com,2019-05-20
2,3,Carlos Oliveira,Atendente,carlos.oliveira@clinica.com,2021-03-10
3,4,Fernanda Lima,Recepcionista,fernanda.lima@clinica.com,2022-07-01
4,5,Ricardo Santos,Auxiliar Veterinário,ricardo.santos@clinica.com,2023-02-12


In [ ]:
consulta('''
SELECT * FROM Consultas;
''','')

,id,data_consulta,id_animal,id_funcionario,id_servico,observacoes,id_veterinario
0,1,2023-10-25 10:00:00,1,1,1,Animal apresentou febre e falta de apetite.,1
1,2,2023-10-26 14:30:00,2,2,2,Aplicação da vacina V8.,2
2,3,2023-10-27 09:15:00,3,1,3,Cirurgia de castração realizada com sucesso.,1
3,4,2023-10-28 11:00:00,4,2,4,Exame de sangue para check-up anual.,2
4,5,2023-10-29 16:45:00,5,3,5,Banho e tosa completo.,3


In [ ]:
consulta('''
SELECT * FROM Tratamentos;
''','')

,id,id_consulta,medicamento,dosagem,frequencia,duracao
0,1,1,Doxiciclina,1 comprimido,A cada 12 horas,7 dias
1,2,2,Vacina V8,1 dose,Única,1 dia
2,3,3,Metadona,1 comprimido,A cada 8 horas,3 dias
3,4,4,Promun dog tabs,1 comprimido,Diário,30 dias
4,5,5,Shampoo de tratamento,Uso tópico,Durante o banho,30 dias


In [ ]:
consulta('''
SELECT * FROM Pagamentos;
''','')

,id,id_consulta,valor,data_pagamento,metodo_pagamento
0,1,1,150.00,2023-10-25,Cartão de Crédito
1,2,2,80.00,2023-10-26,Dinheiro
2,3,3,500.00,2023-10-27,PIX
3,4,4,120.00,2023-10-28,Cartão de Débito
4,5,5,60.00,2023-10-29,Dinheiro


In [ ]:
consulta('''
SELECT * FROM Tutores;
''','')

,id,nome,telefone,email,endereco,data_cadastro
0,1,Ana Clara,(32) 9999-8888,ana.clara@email.com,"Rua das Flores, 123",2023-01-10
1,2,Carlos Eduardo,(32) 7777-6666,carlos.eduardo@email.com,"Avenida Paulista, 456",2023-02-15
2,3,Mariana Silva,(32) 5555-4444,mariana.silva@email.com,"Rua dos Pinheiros, 789",2023-03-20
3,4,Pedro Henrique,(32) 3333-2222,pedro.henrique@email.com,"Alameda Santos, 101",2023-04-25
4,5,Juliana Costa,(32) 1111-0000,juliana.costa@email.com,"Rua Augusta, 202",2023-05-30


In [ ]:
consulta('''
SELECT * FROM Servicos;
''','')

,id,nome,descricao,preco
0,1,Consulta Geral,Consulta de rotina para avaliação de saúde,150.00
1,2,Vacinação,Aplicação de vacinas,80.00
2,3,Cirurgia,Procedimento cirúrgico,500.00
3,4,Exame de Sangue,Coleta e análise de sangue,120.00
4,5,Banho e Tosa,Serviço de higiene e estética,60.00


Modificação de tabelas (ALTER TABLE).

In [ ]:
ddl('''
# Adicionar uma nova coluna para CPF no cadastro de Tutores
ALTER TABLE Tutores ADD COLUMN cpf VARCHAR(14) UNIQUE;
''')

ddl('''
# Tornar o email de Funcionarios um campo obrigatório
ALTER TABLE Funcionarios MODIFY COLUMN email VARCHAR(100) NOT NULL;
''')

ddl('''
# Adicionar um campo para armazenar a especialidade dos veterinários na tabela Funcionarios
ALTER TABLE Funcionarios ADD COLUMN especialidade VARCHAR(100) NULL;
''')

ddl('''
# Adicionar uma restrição para garantir que o preço dos serviços não seja negativo
ALTER TABLE Servicos MODIFY COLUMN preco DECIMAL(10,2) NOT NULL CHECK (preco >= 0);
''')

ddl('''
# Modificar a coluna peso da tabela Animais para aceitar até 3 casas decimais
ALTER TABLE Animais MODIFY COLUMN peso DECIMAL(6,3);
''')

ddl('''
# Adicionar um campo para registrar a duração da consulta na tabela Consultas
ALTER TABLE Consultas ADD COLUMN duracao INT COMMENT 'Duração em minutos';
''')

ddl('''
# Adicionar um campo para indicar se o pagamento foi confirmado na tabela Pagamentos
ALTER TABLE Pagamentos ADD COLUMN confirmado BOOLEAN DEFAULT FALSE;
''')

ddl('''
# Adicionar um campo para observações extras no tratamento
ALTER TABLE Tratamentos ADD COLUMN observacoes TEXT;
''')


Erro ao executar o comando DDL: 1060 (42S21): Duplicate column name 'cpf'
Comando DDL executado com sucesso:

# Tornar o email de Funcionarios um campo obrigatório
ALTER TABLE Funcionarios MODIFY COLUMN email VARCHAR(100) NOT NULL;

Comando DDL executado com sucesso:

# Adicionar um campo para armazenar a especialidade dos veterinários na tabela Funcionarios
ALTER TABLE Funcionarios ADD COLUMN especialidade VARCHAR(100) NULL;

Comando DDL executado com sucesso:

# Adicionar uma restrição para garantir que o preço dos serviços não seja negativo
ALTER TABLE Servicos MODIFY COLUMN preco DECIMAL(10,2) NOT NULL CHECK (preco >= 0);

Comando DDL executado com sucesso:

# Modificar a coluna peso da tabela Animais para aceitar até 3 casas decimais
ALTER TABLE Animais MODIFY COLUMN peso DECIMAL(6,3);

Comando DDL executado com sucesso:

# Adicionar um campo para registrar a duração da consulta na tabela Consultas
ALTER TABLE Consultas ADD COLUMN duracao INT COMMENT 'Duração em minutos';

Erro ao

Exclusão de tabelas ou colunas (DROP)

In [ ]:
ddl('''
# Remover a coluna "especialidade" da tabela Funcionarios
ALTER TABLE Funcionarios DROP COLUMN especialidade;
''')

ddl('''
# Remover a coluna "confirmado" da tabela Pagamentos
ALTER TABLE Pagamentos DROP COLUMN confirmado;
''')

ddl('''
# Remover a coluna "duracao" da tabela Consultas
ALTER TABLE Consultas DROP COLUMN duracao;
''')

ddl('''
# Excluir a tabela Tratamentos
DROP TABLE Tratamentos;
''')

ddl('''
# Excluir a tabela Pagamentos
DROP TABLE Pagamentos;
''')


Comando DDL executado com sucesso:

# Remover a coluna "especialidade" da tabela Funcionarios
ALTER TABLE Funcionarios DROP COLUMN especialidade;

Comando DDL executado com sucesso:

# Remover a coluna "confirmado" da tabela Pagamentos
ALTER TABLE Pagamentos DROP COLUMN confirmado;

Comando DDL executado com sucesso:

# Remover a coluna "duracao" da tabela Consultas
ALTER TABLE Consultas DROP COLUMN duracao;

Comando DDL executado com sucesso:

# Excluir a tabela Tratamentos
DROP TABLE Tratamentos;

Comando DDL executado com sucesso:

# Excluir a tabela Pagamentos
DROP TABLE Pagamentos;



Exclusão de registros (DELETE)

In [ ]:
consulta('''
SELECT * FROM Tutores;
''','')

,id,nome,telefone,email,endereco,data_cadastro,cpf
0,1,Ana Clara,(32) 9999-7777,ana.clara@email.com,"Rua das Flores, 123",2023-01-10,None
1,2,Carlos Eduardo,(32) 7777-6666,carlos.eduardo@email.com,"Avenida Paulista, 456",2023-02-15,None
2,3,Mariana Silva,(32) 5555-4444,mariana.silva@email.com,"Rua dos Pinheiros, 789",2023-03-20,None
3,4,Pedro Henrique,(32) 3333-2222,pedro.henrique@email.com,"Alameda Santos, 101",2023-04-25,None
4,5,Juliana Costa,(32) 1111-0000,juliana.costa@email.com,"Rua Augusta, 202",2023-05-30,None


In [ ]:
consulta('''
DESCRIBE Animais;
''','')

,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,
1,nome,varchar(100),NO,,None,
2,especie,varchar(50),NO,,None,
3,raca,varchar(50),YES,,None,
4,data_nascimento,date,YES,,None,
5,peso,"decimal(6,3)",YES,,None,
6,id_tutor,int,YES,MUL,None,


In [ ]:
ddl('''
DROP TABLE Tutores;
''')



Erro ao executar o comando DDL: 3730 (HY000): Cannot drop table 'Tutores' referenced by a foreign key constraint 'Animais_ibfk_1' on table 'Animais'.


In [ ]:
ddl('''
ALTER TABLE Animais DROP id_tutor;

ALTER TABLE Animais
ADD CONSTRAINT id_tutor
FOREIGN KEY (id_tutor) REFERENCES Tutores(id)
ON DELETE CASCADE;
''')

#alterando o id_tutor e deletar em cascade

Erro ao executar o comando DDL: 1828 (HY000): Cannot drop column 'id_tutor': needed in a foreign key constraint 'Animais_ibfk_1'


In [ ]:
ddl('''
ALTER TABLE Animais DROP FOREIGN KEY id_tutor;
''')

Comando DDL executado com sucesso:

ALTER TABLE Animais DROP FOREIGN KEY id_tutor;



In [ ]:
#excluir tutore 2 e 4
ddl('''
DELETE FROM Tutores WHERE id IN(2,4);
''')


Erro ao executar o comando DDL: 1451 (23000): Cannot delete or update a parent row: a foreign key constraint fails (`clinica_veterinaria`.`Animais`, CONSTRAINT `Animais_ibfk_1` FOREIGN KEY (`id_tutor`) REFERENCES `Tutores` (`id`))


In [ ]:
consulta('''
SELECT * FROM Tutores;
''','')

,id,nome,telefone,email,endereco,data_cadastro,cpf
0,1,Ana Clara,(32) 9999-7777,ana.clara@email.com,"Rua das Flores, 123",2023-01-10,None
1,2,Carlos Eduardo,(32) 7777-6666,carlos.eduardo@email.com,"Avenida Paulista, 456",2023-02-15,None
2,3,Mariana Silva,(32) 5555-4444,mariana.silva@email.com,"Rua dos Pinheiros, 789",2023-03-20,None
3,4,Pedro Henrique,(32) 3333-2222,pedro.henrique@email.com,"Alameda Santos, 101",2023-04-25,None
4,5,Juliana Costa,(32) 1111-0000,juliana.costa@email.com,"Rua Augusta, 202",2023-05-30,None


In [ ]:
consulta('''SELECT TABLE_NAME, COLUMN_NAME, CONSTRAINT_NAME, REFERENCED_TABLE_NAME, REFERENCED_COLUMN_NAME
FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE
WHERE REFERENCED_TABLE_NAME = 'Tutores';
''','')

,TABLE_NAME,COLUMN_NAME,CONSTRAINT_NAME,REFERENCED_TABLE_NAME,REFERENCED_COLUMN_NAME
0,Animais,id_tutor,Animais_ibfk_1,Tutores,id
1,Animais,id_tutor,id_tutor,Tutores,id


In [ ]:
consulta('''SELECT TABLE_NAME, COLUMN_NAME, CONSTRAINT_NAME, REFERENCED_TABLE_NAME, REFERENCED_COLUMN_NAME
FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE
WHERE REFERENCED_TABLE_NAME = 'Animais';
''','')

,TABLE_NAME,COLUMN_NAME,CONSTRAINT_NAME,REFERENCED_TABLE_NAME,REFERENCED_COLUMN_NAME
0,Consultas,id_animal,Consultas_ibfk_1,Animais,id


In [ ]:
consulta('''
DESCRIBE Animais;
''','')

,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,
1,nome,varchar(100),NO,,None,
2,especie,varchar(50),NO,,None,
3,raca,varchar(50),YES,,None,
4,data_nascimento,date,YES,,None,
5,peso,"decimal(6,3)",YES,,None,
6,id_tutor,int,YES,MUL,None,


Atualização de registros (UPDATE)

In [ ]:
ddl('''
# Atualizar o telefone de um tutor específico
UPDATE Tutores
SET telefone = '(32) 9999-7777'
WHERE id = 1;

# Atualizar o peso de um animal
UPDATE Animais
SET peso = 32.0
WHERE id = 1;

# Atualizar o cargo de um funcionário
UPDATE Funcionarios
SET cargo = 'Gerente'
WHERE id = 3;

# Atualizar o preço de um serviço
UPDATE Servicos
SET preco = 180.00
WHERE id = 1;

# Atualizar a observação de uma consulta
UPDATE Consultas
SET observacoes = 'Animal em recuperação, retorno em 7 dias.'
WHERE id = 3;
''')


Comando DDL executado com sucesso:

# Atualizar o telefone de um tutor específico
UPDATE Tutores 
SET telefone = '(32) 9999-7777' 
WHERE id = 1;

# Atualizar o peso de um animal
UPDATE Animais 
SET peso = 32.0 
WHERE id = 1;

# Atualizar o cargo de um funcionário
UPDATE Funcionarios 
SET cargo = 'Gerente' 
WHERE id = 3;

# Atualizar o preço de um serviço
UPDATE Servicos 
SET preco = 180.00 
WHERE id = 1;

# Atualizar a observação de uma consulta
UPDATE Consultas 
SET observacoes = 'Animal em recuperação, retorno em 7 dias.' 
WHERE id = 3;



SELECT * de todas tabelas, para mostrar todos os registros criados.

In [ ]:
consulta('''
SELECT * FROM Animais;
''','')

,id,nome,especie,raca,data_nascimento,peso,id_tutor
0,1,Rex,Cachorro,Labrador,2018-05-10,32.000,1
1,2,Mimi,Gato,Siamês,2019-03-15,4.200,2
2,3,Thor,Cachorro,Bulldog,2020-07-22,25.000,3
3,4,Luna,Gato,Persa,2017-11-30,5.000,4
4,5,Bella,Cachorro,Poodle,2021-01-05,8.700,5


In [ ]:
consulta('''
SELECT * FROM Funcionarios;
''','')

,id,nome,cargo,email,data_admissao,especialidade
0,1,Dr. João Silva,Veterinário,joao.silva@clinica.com,2020-01-15,None
1,2,Dra. Maria Souza,Veterinária,maria.souza@clinica.com,2019-05-20,None
2,3,Carlos Oliveira,Gerente,carlos.oliveira@clinica.com,2021-03-10,None
3,5,Ricardo Santos,Auxiliar Veterinário,ricardo.santos@clinica.com,2023-02-12,None


In [ ]:
consulta('''
SELECT * FROM Consultas;
''','')

,id,data_consulta,id_animal,id_funcionario,id_servico,observacoes,id_veterinario,duracao
0,1,2023-10-25 10:00:00,1,1,1,Animal apresentou febre e falta de apetite.,1,None
1,2,2023-10-26 14:30:00,2,2,2,Aplicação da vacina V8.,2,None
2,4,2023-10-28 11:00:00,4,2,4,Exame de sangue para check-up anual.,2,None
3,5,2023-10-29 16:45:00,5,3,5,Banho e tosa completo.,3,None


In [ ]:
consulta('''
SELECT * FROM Tratamentos;
''','')

,id,id_consulta,medicamento,dosagem,frequencia,duracao,observacoes
0,1,1,Doxiciclina,1 comprimido,A cada 12 horas,7 dias,None
1,2,2,Vacina V8,1 dose,Única,1 dia,None
2,3,3,Metadona,1 comprimido,A cada 8 horas,3 dias,None
3,4,4,Promun dog tabs,1 comprimido,Diário,30 dias,None
4,5,5,Shampoo de tratamento,Uso tópico,Durante o banho,30 dias,None


In [ ]:
consulta('''
SELECT * FROM Pagamentos;
''','')

,id,id_consulta,valor,data_pagamento,metodo_pagamento,confirmado
0,1,1,150.00,2023-10-25,Cartão de Crédito,0
1,2,2,80.00,2023-10-26,Dinheiro,0
2,3,3,500.00,2023-10-27,PIX,0
3,4,4,120.00,2023-10-28,Cartão de Débito,0
4,5,5,60.00,2023-10-29,Dinheiro,0


In [ ]:
consulta('''
SELECT * FROM Tutores;
''','')

,id,nome,telefone,email,endereco,data_cadastro,cpf
0,1,Ana Clara,(32) 9999-7777,ana.clara@email.com,"Rua das Flores, 123",2023-01-10,None
1,2,Carlos Eduardo,(32) 7777-6666,carlos.eduardo@email.com,"Avenida Paulista, 456",2023-02-15,None
2,3,Mariana Silva,(32) 5555-4444,mariana.silva@email.com,"Rua dos Pinheiros, 789",2023-03-20,None
3,4,Pedro Henrique,(32) 3333-2222,pedro.henrique@email.com,"Alameda Santos, 101",2023-04-25,None
4,5,Juliana Costa,(32) 1111-0000,juliana.costa@email.com,"Rua Augusta, 202",2023-05-30,None


In [ ]:
consulta('''
SELECT * FROM Servicos;
''','')

,id,nome,descricao,preco
0,1,Consulta Geral,Consulta de rotina para avaliação de saúde,180.00
1,2,Vacinação,Aplicação de vacinas,80.00
2,3,Cirurgia,Procedimento cirúrgico,500.00
3,4,Exame de Sangue,Coleta e análise de sangue,120.00
4,5,Banho e Tosa,Serviço de higiene e estética,60.00


DROP EM TUDO, TODAS AS TABELAS

In [ ]:
ddl('''
DROP TABLE Consultas
''')

Comando DDL executado com sucesso:

DROP TABLE Consultas



In [ ]:
ddl('''
DROP TABLE Animais;
''')

Comando DDL executado com sucesso:

DROP TABLE Animais;



In [ ]:
ddl('''
DROP TABLE Servicos;
''')

Comando DDL executado com sucesso:

DROP TABLE Servicos;



In [ ]:
ddl('''
DROP TABLE Tutores;

''')

Comando DDL executado com sucesso:

DROP TABLE Tutores;




In [ ]:
ddl('''
DROP TABLE Funcionarios;
''')

Comando DDL executado com sucesso:

DROP TABLE Funcionarios;



In [ ]:
consulta('''
SHOW TABLES;
''','')

,Tables_in_clinica_veterinaria


Criando todas as tabelas novamente

In [ ]:
ddl('''
CREATE TABLE Tutores (
    id INT PRIMARY KEY,
    nome VARCHAR(100) NOT NULL,
    telefone VARCHAR(15),
    email VARCHAR(100) UNIQUE,
    endereco VARCHAR(200),
    data_cadastro DATE
);
''')

ddl('''
CREATE TABLE Animais (
    id INT PRIMARY KEY,
    nome VARCHAR(100) NOT NULL,
    especie VARCHAR(50) NOT NULL,
    raca VARCHAR(50),
    data_nascimento DATE,
    peso DECIMAL(5, 2),
    id_tutor INT,
    FOREIGN KEY (id_tutor) REFERENCES Tutores(id) ON DELETE SET NULL
);
''')

ddl('''
CREATE TABLE Funcionarios (
    id INT PRIMARY KEY,
    nome VARCHAR(100) NOT NULL,
    cargo VARCHAR(50) NOT NULL,
    email VARCHAR(100) UNIQUE,
    data_admissao DATE
);
''')

ddl('''
CREATE TABLE Servicos (
    id INT PRIMARY KEY,
    nome VARCHAR(100) NOT NULL,
    descricao TEXT,
    preco DECIMAL(10, 2) NOT NULL
);
''')

ddl('''
CREATE TABLE Consultas (
    id INT PRIMARY KEY,
    data_consulta DATETIME NOT NULL,
    id_animal INT,
    id_funcionario INT,
    id_servico INT,
    observacoes VARCHAR(500),
    id_veterinario INT,
    FOREIGN KEY (id_animal) REFERENCES Animais(id) ON DELETE SET NULL,
    FOREIGN KEY (id_funcionario) REFERENCES Funcionarios(id) ON DELETE SET NULL,
    FOREIGN KEY (id_servico) REFERENCES Servicos(id) ON DELETE SET NULL,
    FOREIGN KEY (id_veterinario) REFERENCES Funcionarios(id) ON DELETE SET NULL
);
''')

ddl('''
CREATE TABLE Tratamentos (
    id INT PRIMARY KEY,
    id_consulta INT,
    medicamento VARCHAR(100) NOT NULL,
    dosagem VARCHAR(50),
    frequencia VARCHAR(50),
    duracao VARCHAR(50),
    FOREIGN KEY (id_consulta) REFERENCES Consultas(id) ON DELETE SET NULL
);
''')

ddl('''
CREATE TABLE Pagamentos (
    id INT PRIMARY KEY,
    id_consulta INT,
    valor DECIMAL(10, 2) NOT NULL,
    data_pagamento DATE,
    metodo_pagamento VARCHAR(50),
    FOREIGN KEY (id_consulta) REFERENCES Consultas(id) ON DELETE SET NULL
);
''')

Comando DDL executado com sucesso:

CREATE TABLE Tutores (
    id INT PRIMARY KEY,
    nome VARCHAR(100) NOT NULL,
    telefone VARCHAR(15),
    email VARCHAR(100) UNIQUE,
    endereco VARCHAR(200),
    data_cadastro DATE
);

Comando DDL executado com sucesso:

CREATE TABLE Animais (
    id INT PRIMARY KEY,
    nome VARCHAR(100) NOT NULL,
    especie VARCHAR(50) NOT NULL,
    raca VARCHAR(50),
    data_nascimento DATE,
    peso DECIMAL(5, 2),
    id_tutor INT,
    FOREIGN KEY (id_tutor) REFERENCES Tutores(id) ON DELETE SET NULL
);

Comando DDL executado com sucesso:

CREATE TABLE Funcionarios (
    id INT PRIMARY KEY,
    nome VARCHAR(100) NOT NULL,
    cargo VARCHAR(50) NOT NULL,
    email VARCHAR(100) UNIQUE,
    data_admissao DATE
);

Comando DDL executado com sucesso:

CREATE TABLE Servicos (
    id INT PRIMARY KEY,
    nome VARCHAR(100) NOT NULL,
    descricao TEXT,
    preco DECIMAL(10, 2) NOT NULL
);

Comando DDL executado com sucesso:

CREATE TABLE Consultas (
    id INT PRIMAR

Inserindo todos os dados novamente

In [ ]:
# Inserir Tutores
ddl('''
INSERT INTO Tutores (id, nome, telefone, email, endereco, data_cadastro) VALUES
(1, 'Ana Clara', '(32) 9999-8888', 'ana.clara@email.com', 'Rua das Flores, 123', '2023-01-10'),
(2, 'Carlos Eduardo', '(32) 7777-6666', 'carlos.eduardo@email.com', 'Avenida Paulista, 456', '2023-02-15'),
(3, 'Mariana Silva', '(32) 5555-4444', 'mariana.silva@email.com', 'Rua dos Pinheiros, 789', '2023-03-20'),
(4, 'Pedro Henrique', '(32) 3333-2222', 'pedro.henrique@email.com', 'Alameda Santos, 101', '2023-04-25'),
(5, 'Juliana Costa', '(32) 1111-0000', 'juliana.costa@email.com', 'Rua Augusta, 202', '2023-05-30');
''')

# Inserir Animais
ddl('''
INSERT INTO Animais (id, nome, especie, raca, data_nascimento, peso, id_tutor) VALUES
(1, 'Rex', 'Cachorro', 'Labrador', '2018-05-10', 30.5, 1),
(2, 'Mimi', 'Gato', 'Siamês', '2019-03-15', 4.2, 2),
(3, 'Thor', 'Cachorro', 'Bulldog', '2020-07-22', 25.0, 3),
(4, 'Luna', 'Gato', 'Persa', '2017-11-30', 5.0, 4),
(5, 'Bella', 'Cachorro', 'Poodle', '2021-01-05', 8.7, 5);
''')

# Inserir Funcionários
ddl('''
INSERT INTO Funcionarios (id, nome, cargo, email, data_admissao) VALUES
(1, 'Dr. João Silva', 'Veterinário', 'joao.silva@clinica.com', '2020-01-15'),
(2, 'Dra. Maria Souza', 'Veterinária', 'maria.souza@clinica.com', '2019-05-20'),
(3, 'Carlos Oliveira', 'Atendente', 'carlos.oliveira@clinica.com', '2021-03-10'),
(4, 'Fernanda Lima', 'Recepcionista', 'fernanda.lima@clinica.com', '2022-07-01'),
(5, 'Ricardo Santos', 'Auxiliar Veterinário', 'ricardo.santos@clinica.com', '2023-02-12');
''')

# Inserir Serviços
ddl('''
INSERT INTO Servicos (id, nome, descricao, preco) VALUES
(1, 'Consulta Geral', 'Consulta de rotina para avaliação de saúde', 150.00),
(2, 'Vacinação', 'Aplicação de vacinas', 80.00),
(3, 'Cirurgia', 'Procedimento cirúrgico', 500.00),
(4, 'Exame de Sangue', 'Coleta e análise de sangue', 120.00),
(5, 'Banho e Tosa', 'Serviço de higiene e estética', 60.00);
''')

# Inserir Consultas
ddl('''
INSERT INTO Consultas (id, data_consulta, id_animal, id_funcionario, id_servico, observacoes, id_veterinario) VALUES
(1, '2023-10-25 10:00:00', 1, 1, 1, 'Animal apresentou febre e falta de apetite.', 1),
(2, '2023-10-26 14:30:00', 2, 2, 2, 'Aplicação da vacina V8.', 2),
(3, '2023-10-27 09:15:00', 3, 1, 3, 'Cirurgia de castração realizada com sucesso.', 1),
(4, '2023-10-28 11:00:00', 4, 2, 4, 'Exame de sangue para check-up anual.', 2),
(5, '2023-10-29 16:45:00', 5, 3, 5, 'Banho e tosa completo.', 3);
''')

# Inserir Tratamentos
ddl('''
INSERT INTO Tratamentos (id, id_consulta, medicamento, dosagem, frequencia, duracao) VALUES
(1, 1, 'Doxiciclina', '1 comprimido', 'A cada 12 horas', '7 dias'),
(2, 2, 'Vacina V8', '1 dose', 'Única', '1 dia'),
(3, 3, 'Metadona', '1 comprimido', 'A cada 8 horas', '3 dias'),
(4, 4, 'Promun dog tabs', '1 comprimido', 'Diário', '30 dias'),
(5, 5, 'Shampoo de tratamento', 'Uso tópico', 'Durante o banho', '30 dias');
''')

# Inserir Pagamentos
ddl('''
INSERT INTO Pagamentos (id, id_consulta, valor, data_pagamento, metodo_pagamento) VALUES
(1, 1, 150.00, '2023-10-25', 'Cartão de Crédito'),
(2, 2, 80.00, '2023-10-26', 'Dinheiro'),
(3, 3, 500.00, '2023-10-27', 'PIX'),
(4, 4, 120.00, '2023-10-28', 'Cartão de Débito'),
(5, 5, 60.00, '2023-10-29', 'Dinheiro');
''')


Comando DDL executado com sucesso:

INSERT INTO Tutores (id, nome, telefone, email, endereco, data_cadastro) VALUES
(1, 'Ana Clara', '(32) 9999-8888', 'ana.clara@email.com', 'Rua das Flores, 123', '2023-01-10'),
(2, 'Carlos Eduardo', '(32) 7777-6666', 'carlos.eduardo@email.com', 'Avenida Paulista, 456', '2023-02-15'),
(3, 'Mariana Silva', '(32) 5555-4444', 'mariana.silva@email.com', 'Rua dos Pinheiros, 789', '2023-03-20'),
(4, 'Pedro Henrique', '(32) 3333-2222', 'pedro.henrique@email.com', 'Alameda Santos, 101', '2023-04-25'),
(5, 'Juliana Costa', '(32) 1111-0000', 'juliana.costa@email.com', 'Rua Augusta, 202', '2023-05-30');

Comando DDL executado com sucesso:

INSERT INTO Animais (id, nome, especie, raca, data_nascimento, peso, id_tutor) VALUES
(1, 'Rex', 'Cachorro', 'Labrador', '2018-05-10', 30.5, 1),
(2, 'Mimi', 'Gato', 'Siamês', '2019-03-15', 4.2, 2),
(3, 'Thor', 'Cachorro', 'Bulldog', '2020-07-22', 25.0, 3),
(4, 'Luna', 'Gato', 'Persa', '2017-11-30', 5.0, 4),
(5, 'Bella', 'Cachor

DELETE tutores 2 e 4

In [ ]:
#excluir tutores 2 e 4
ddl('''
DELETE FROM Tutores WHERE id IN(2,4);
''')

Comando DDL executado com sucesso:

DELETE FROM Tutores WHERE id IN(2,4);



SELECT * de todas as tabelas

In [ ]:
consulta('''
SELECT * FROM Tutores;
''','')

,id,nome,telefone,email,endereco,data_cadastro,cpf
0,1,Ana Clara,(32) 9999-7777,ana.clara@email.com,"Rua das Flores, 123",2023-01-10,None
1,3,Mariana Silva,(32) 5555-4444,mariana.silva@email.com,"Rua dos Pinheiros, 789",2023-03-20,None
2,5,Juliana Costa,(32) 1111-0000,juliana.costa@email.com,"Rua Augusta, 202",2023-05-30,None


In [ ]:
consulta('''
SELECT * FROM Animais;
''','')

,id,nome,especie,raca,data_nascimento,peso,id_tutor
0,1,Rex,Cachorro,Labrador,2018-05-10,32.000,1.0
1,2,Mimi,Gato,Siamês,2019-03-15,4.200,NaN
2,3,Thor,Cachorro,Bulldog,2020-07-22,25.000,3.0
3,4,Luna,Gato,Persa,2017-11-30,5.000,NaN
4,5,Bella,Cachorro,Poodle,2021-01-05,8.700,5.0


In [ ]:
consulta('''
SELECT * FROM Funcionarios;
''','')

,id,nome,cargo,email,data_admissao,especialidade
0,1,Dr. João Silva,Veterinário,joao.silva@clinica.com,2020-01-15,None
1,2,Dra. Maria Souza,Veterinária,maria.souza@clinica.com,2019-05-20,None
2,3,Carlos Oliveira,Gerente,carlos.oliveira@clinica.com,2021-03-10,None
3,4,Fernanda Lima,Recepcionista,fernanda.lima@clinica.com,2022-07-01,None
4,5,Ricardo Santos,Auxiliar Veterinário,ricardo.santos@clinica.com,2023-02-12,None


In [ ]:
consulta('''
SELECT * FROM Servicos;
''','')

,id,nome,descricao,preco
0,1,Consulta Geral,Consulta de rotina para avaliação de saúde,180.00
1,2,Vacinação,Aplicação de vacinas,80.00
2,3,Cirurgia,Procedimento cirúrgico,500.00
3,4,Exame de Sangue,Coleta e análise de sangue,120.00
4,5,Banho e Tosa,Serviço de higiene e estética,60.00


In [ ]:
consulta('''
SELECT * FROM Tratamentos;
''','')

ProgrammingError: 1146 (42S02): Table 'clinica_veterinaria.Tratamentos' doesn't exist

In [ ]:
consulta('''
SELECT * FROM Consultas;
''','')

,id,data_consulta,id_animal,id_funcionario,id_servico,observacoes,id_veterinario,duracao
0,1,2023-10-25 10:00:00,1,1,1,Animal apresentou febre e falta de apetite.,1,None
1,2,2023-10-26 14:30:00,2,2,2,Aplicação da vacina V8.,2,None
2,3,2023-10-27 09:15:00,3,1,3,"Animal em recuperação, retorno em 7 dias.",1,None
3,4,2023-10-28 11:00:00,4,2,4,Exame de sangue para check-up anual.,2,None
4,5,2023-10-29 16:45:00,5,3,5,Banho e tosa completo.,3,None


In [ ]:
consulta('''
SELECT * FROM Pagamentos;
''','')

ProgrammingError: 1146 (42S02): Table 'clinica_veterinaria.Pagamentos' doesn't exist